# **Implementación de Turning Grille en Python**
_Juan Esteban Alarcón Bravo_

## 1. Grilla y Mensaje

En el caso de _Turning Grille_ no se usa la usual llave de texto sino que la grilla, una matriz cuadrada con agujeros que se rota en un sentido arbitrario, actúa como la "llave". Se inicia entonces el procedimiento solicitando al usuario el ***mensaje*** (`msg`) y las especificaciones de la grilla (Su ***dimensión*** `n`, el ***número de agujeros*** `x` y el ***sentido de rotación*** `x`). El mensaje se pone todo en mayúsculas, se eliminan los caracteres de espacio (en caso de tenerlos) y se hace padding con $X$ en caso de ser necesario:

In [1]:
import numpy as np

# Mensaje
msg = input('\nIngrese el mensaje: ')
msg = msg.replace(" ", "")
msg = msg.upper()

# Parámetros de la grilla
n = int(input('\nIngrese el tamaño (n) de la grilla: '))
h = int(input('Ingrese la cantidad de huecos: '))

# Grilla ("Llave")
K = np.zeros((n, n))

# Posiciones de los huecos
print('Ingrese la posición (i,j) de cada hueco:')
for num in range(0,h):
  print('--------')
  i = int(input('i:'))
  j = int(input('j:'))
  K[i, j] = 1;

# Previsualización de la grilla
print('\nGrilla:')
print(K)

# Sentido de rotación
rot = int(input('\nIngrese el sentido de rotación:\n 1. Izquierda \n 2. Derecha \n'))

# Padding con 'X'
while len(msg) < n*n:
  msg += 'X'


Ingrese el mensaje: I Need a lot of Rest

Ingrese el tamaño (n) de la grilla: 4
Ingrese la cantidad de huecos: 4
Ingrese la posición (i,j) de cada hueco:
--------
i:0
j:0
--------
i:2
j:1
--------
i:3
j:2
--------
i:2
j:3

Grilla:
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 1.]
 [0. 0. 1. 0.]]

Ingrese el sentido de rotación:
 1. Izquierda 
 2. Derecha 
1


## 2. Inicialización de Diccionarios y Definición de Funciones Auxiliares

_**Diccionarios:**_

Suponemos que se está usando un alfabeto de 26 caracteres ($ABCDEFGHIJKLMNOPQRSTUVWXYZ$).

* `L2N` ("Letter to Number") permite mapear cada letra mayúscula a un número entero.
* `N2L` ("Number to Letter") es el opuesto de `L2N`: a cada número le asigna una letra mayúscula.

In [2]:
# Diccionario: Letras a Números
L2N = {
  'A': 0,  'B': 1,  'C': 2,  'D': 3,  'E': 4,  'F': 5,  'G': 6,  'H': 7,  
  'I': 8,  'J': 9,  'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 
  'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23,
  'Y': 24, 'Z': 25
}
  
# Diccionario: Números a Letras
N2L = {v: k for k, v in L2N.items()}

_**Funciones:**_
* `matrixLeft()` y `matrixRight()` permiten rotar una matriz $K_{n \times n}$ a la izquierda y a la derecha, respectivamente. Esta matriz $K$ es la grilla.
* `msgToMatrix()` permite convertir un mensaje de string a matriz cuadrada de tamaño `n` para su tratamiento en el algoritmo ($msg \rightarrow M_{n \times n}$).
* `getValues()` permite recuperar las letras de la matriz $M$ en las posiciones "visibles" a través de la grilla $K$ haciendo uso del diccionario `N2L`.
* `hideValues()` hace un proceso inverso: oculta las letras usando el diccionario `L2N`.

In [3]:
def matrixLeft(K,n):
  O =  np.zeros((n, n))
  for i in range(0,n):
    for j in range(0,n):
      O[n-(j+1)][i] = K[i][j]
  return O

def matrixRight(K,n):
  O =  np.zeros((n, n))
  for i in range(0,n):
    for j in range(0,n):
      O[j][n-(i+1)] = K[i][j]
  return O


def msgToMatrix(msg,n):
  M = np.zeros((n, n))
  for l in range(0, len(msg)):
    i = (int(l/n))
    j = (int(l%n))
    M[i][j] = L2N[msg[l]]
  return M

def getValues(M,K,n):
  out = ""
  for i in range(0,n):
    for j in range(0,n):
      if K[i][j] == 1:
        out += N2L[M[i][j]]
  return out
 
def hideValues(l,K,n,M):
  count = 0
  for i in range(0,n):
    for j in range(0,n):
      if K[i][j] == 1:
        M[i][j] = L2N[l[count]]
        count += 1
  return M

## 3. Encripción y Desencripción

Para el proceso de encripción se ubican las letras del mensaje `msg` en la matriz $M$ siguiendo la guía de los huecos de la grilla $K$ y el sentido de rotación `rot`. El mensaje cifrado será entonces la matriz $M$ leída fila por fila, de arriba hacia abajo.

In [4]:
def encrypt(msg,K,n,h,rot):
  M = np.zeros((n, n))
  for x in range(0, 4):
    l1 = int((len(msg)/h) * x)
    l2 = int((len(msg)/h) * x) + h
    
    l = msg[l1:l2]

    M = hideValues(l, K, n, M)
    if rot == 1:
      K = matrixLeft(K, n)
    if rot == 2:
      K = matrixRight(K, n)
      
  out = ""
  for i in range(0,n):
    for j in range(0,n):
      out += N2L[M[i][j]]

  return out

Para desencriptar, se inicia poniendo el mensaje `msg` en forma de matriz ($M$). Luego, se comienzan a recuperar los valores que marcan los huecos de la grilla $K$ (con la función `getValues()`); por último, se rota la grilla y se repite el proceso. El resultado final (`out`) será la concatenación de todas las letras recuperadas, en orden.

In [5]:
def decrypt(msg,K,n,h,rot):
  M = msgToMatrix(msg, n)
  out = ""
  for giro in range(0, 4):
    out += getValues(M,K,n)
    if rot == 1:
      K = matrixLeft(K, n)
    if rot == 2:
      K = matrixRight(K, n)
  
  return out

## 4. Impresión de los resultados

In [6]:
print("TURNING GRILLE \n¿Qué desea hacer?")
choice = int(input(" 1. Encriptar \n 2. Desencriptar \n"))

if choice == 1:
  print("\n---------------\n\nMensaje Original: " + msg)
  print("Grilla: ")
  print(K)
  if rot == 1:
    print("Rotación: Izquierda")
  if rot == 2:
    print("Rotación: Derecha")
      
  out = encrypt(msg,K,n,h,rot)
  print("\n---------------\n\nMensaje Cifrado: " + out)

elif choice == 2:
  print("\n---------------\n\nMensaje Cifrado: " + msg)
  print("Grilla: ")
  print(K)
  if rot == 1:
    print("Rotación: Izquierda")
  if rot == 2:
    print("Rotación: Derecha")

  out = decrypt(msg,K,n,h,rot)
  print("\n---------------\n\nMensaje descifrado: " + out)

TURNING GRILLE 
¿Qué desea hacer?
 1. Encriptar 
 2. Desencriptar 
1

---------------

Mensaje Original: INEEDALOTOFRESTX
Grilla: 
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 1.]
 [0. 0. 1. 0.]]
Rotación: Izquierda

---------------

Mensaje Cifrado: ITDEOSFATNLEOXER
